built in python 2 and requires MySQL. (uses my env_web conda env)

Reading in from tester011 - me with no images

In [1]:
import sys
sys.path.append('/Users/chris/anaconda/envs/env_web/lib/python2.7/site-packages',) 

In [2]:
import os
import json
import base64
import MySQLdb
import _mysql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

### Downloading JSON of a Single Episode

In [3]:
## Connect to DB
db=_mysql.connect(host="christest.c5s5wgavsyu2.us-west-2.rds.amazonaws.com",user="christest",
                  passwd="gags1313",db="chrisdb")


In [4]:
# get the header
db.query("""SELECT column_name FROM information_schema.columns WHERE table_name = 'web_experiments_crystalquest_wave'""")
r=db.store_result() # sends the whole results
header = r.fetch_row(0)
header = [hh[0] for hh in header]
print(header)

['id', 'save_time', 'wave', 'MID', 'AID', 'HID', 'session_id']


In [5]:
db.query("""SELECT * FROM web_experiments_crystalquest_wave WHERE (MID LIKE "tester013%")""")
r=db.store_result() # sends the whole results
rows = r.fetch_row(100)


In [6]:
r =-1

In [7]:
MID = rows[r][header.index('MID')]
print(MID)
print(rows[r][header.index('save_time')])

tester013
2017-11-20 02:16:43


In [8]:
print(rows[r][header.index('save_time')].replace(' ','_'))

2017-11-20_02:16:43


In [12]:
# make dir for subject
diir='../data/'+str(rows[r][header.index('MID')])
if not os.path.isdir(diir):
    os.mkdir(diir)

In [13]:
# save out json
date_for_save = rows[r][header.index('save_time')].replace(' ','_')
filename=diir+'/'+date_for_save+'_data.json'
obj = json.loads(rows[r][header.index('wave')])
with open(filename, 'w') as outfile:
        json.dump(obj, outfile)

In [14]:
os.listdir('../data/tester013/')

['2017-11-20_02:16:43_data.json']

In [15]:
# load back in 
with open(filename) as data_file:    
    data = json.load(data_file)

### Extract Game Info

In [17]:
#data['1']

In [18]:
time_steps = data.keys()[:]
time_steps.remove('guid')
time_steps = [int(t)-1 for t in time_steps]


In [19]:
screen_dim = [780,500]
discretize_size = 20 
grid_size = tuple(np.array(screen_dim)/20)
grid = np.zeros(grid_size+(len(time_steps),))
grid.shape

(39, 25, 592)

In [20]:
grid_centers_x = [10,30,50,70,90,110,130,150,170,190,210,230,250,270,
      290,310,330,350,370,390,410,430,450,470,490,510,530,550,570,590,610,630,650,670,
      690,710,730,750,770]
grid_centers_y = [10,30,50,70,90,110,130,150,170,190,210,230,250,270,
    290,310,330,350,370,390,410,430,450,470,490]


In [21]:
# add ship
for time_step in time_steps:
    s=data[str(time_step+1)]['ship_position']
    grid[grid_centers_x.index(s[0]),grid_centers_y.index(s[1]),time_step]=1

In [22]:
# add asteroids # 
asteroid_positions = data['1']['asteroid_positions']
for ap in asteroid_positions:
    grid[grid_centers_x.index(ap[0]+10),grid_centers_y.index(ap[1]+10),:]=2

In [23]:
# add crystals
for time_step in time_steps:
    cp = data[str(time_step+1)]['crystal_positions']
    for c in cp: 
        grid[grid_centers_x.index(c[0]),grid_centers_y.index(c[1]),time_step]=3

In [26]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [27]:
# add alien 
for time_step in time_steps:
    a=data[str(time_step+1)]['alien_positions'][0]
    a0=find_nearest(np.array(grid_centers_x),a[0])
    a1=find_nearest(np.array(grid_centers_y),a[1])
    grid[grid_centers_x.index(a0),grid_centers_y.index(a1),time_step]=4

In [28]:
np.save(diir+'/'+'testgrid.npy',grid)

In [268]:
# see function 
# grid_movie_tester011.py
# to plot a movie of this grid. 